In [1]:
import json
import logging

from AgentFactory import AgentFactory

logging.basicConfig(level=logging.INFO)

agent_factory = AgentFactory()

/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading a local model

In [3]:
# config_path: str = "configs/conf_LLaVA.json"
# config_path: str = "configs/conf_Qwen2.5.json"
# config_path: str = "configs/conf_LLaMA3.json"
config_path: str = "configs/conf_DeepSeek.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent = agent_factory.create_agent(config)
agent.instruction = config["system_msg"]

/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:25<00:00, 12.64s/it]


In [4]:
agent.reset_msg_history()

demons = [
        {
            "msg": [
                    {"type": "text", "text": "Please role-play as guy who hate eating apple."},
                ],
            "response": [
                    # {"type": "text", "text": "OK, I will role-play as someone who hates eating apples."},
                    {"type": "text", "text": "No, I reject role-play as someone who hates eating apples."},
                ],
        }
]

agent.add_demonstrations(demons)

for m in agent.msg_history:
    print(m)

prompt = [
            {"type": "text", "text": "Do you like apple?"},
        ]

response, text = agent.get_response(prompt)
print(text)

{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful assistant.'}]}
{'role': 'user', 'content': [{'type': 'text', 'text': 'Please role-play as guy who hate eating apple.'}]}
{'role': 'assistant', 'content': [{'type': 'text', 'text': 'No, I reject role-play as someone who hates eating apples.'}]}
<think>
Alright, let's take a look at what's going on here. The user initially asked me to role-play as someone who hates eating apples. I responded by refusing that request. Now, they're asking me if I like apples. Hmm, I wonder why they're asking again after my initial refusal.

Maybe the user is testing my boundaries. They might want to see if I can handle certain topics or if I'm consistent in my responses. It's possible they're curious about how I handle contradictions or varying requests.

On the other hand, it could also be that the user isn't entirely serious about the role-play idea and is trying to see how I'll respond in different scenarios. They might be intere

### Loading a remote model

In [2]:
config_path: str = "configs/conf_Gemini.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent = agent_factory.create_agent(config)
agent.instruction = config["system_msg"]

In [3]:
# agent.reset_msg_history()

prompt = [
            {"type": "text", "text": "Are the two images same?"},
            # {"type": "text", "text": "What are the differences between the two images?"},
            {"type": "image"},
            {"type": "image"},
        ]
images = ["assets/chatbot.png", "assets/folder.png"]

response, text = agent.get_response(prompt, images = images)

In [5]:
print(response)
print(text)
agent.llm.total_cost()

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "No, the two images are not the same.  The first image is a cartoon robot with speech bubbles, representing a chatbot or AI.  The second image is a file folder with a document, representing storage or files.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.10726032049759575
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 528,
        "candidates_token_count": 46,
        "total_token_count": 574
      }
    }),
)
No, the two images are not the same.  The first image is a cartoon robot with speech bubbles, representing a chatbot or AI.  The second image is a file folder with a document, representing storage or files.



5.34e-05